# Neural Networks: Backpropagation of Gradients on Graphs

## Backpropagation of Gradients

- Neural networks are trained by minimization of a loss function $L(\vec{y})$ of outputs $\vec{y}$ 

- In each minimization step, trainable variables must be varied in the opposite direction of the loss gradient $\nabla L$

<img src="fig/example_graph.png" alt="drawing" width="700"/>
How does one vary, for example, trainable parameter $W_i$ in the direction of $-\nabla L$?

$W_{i} \rightarrow W_{i} - \frac{\partial L}{\partial W_{i}}\epsilon$

# Backpropagation of Gradients
Backpropagation on a graph is essentially the chain rule:

$\frac{\partial L}{\partial W_{i}} = \sum_{j}\frac{\partial L}{\partial z_{j}}\sum_{k}\frac{\partial z_{j}}{\partial y_{k}}\frac{\partial y_{k}}{\partial W_{i}} = \sum_{j,k}\frac{\partial L}{\partial z_{j}}\frac{\partial z_{j}}{\partial y_{k}}\frac{\partial y_{k}}{\partial W_{i}} = \sum_{j,k}\frac{\partial y_{k}}{\partial W_{i}}\frac{\partial z_{j}}{\partial y_{k}}\frac{\partial L}{\partial z_{j}}$

$ = \sum_{j,k}\frac{\partial y_{k}}{\partial W_{i}}\Bigl[\frac{\partial z_{j}}{\partial y_{k}}\Bigl(\frac{\partial L}{\partial z_{j}}\Bigr)\Bigr] = \sum_{k}\frac{\partial y_{k}}{\partial W_{i}}\sum_{j}\Bigl[\frac{\partial z_{j}}{\partial y_{k}}dz_{j}\Bigr]$, where $dz_{j} = \frac{\partial L}{\partial z_{j}}$

$ = \sum_{k}\frac{\partial y_{k}}{\partial W_{i}} dy_{k}$, where $dy_{k} = \sum_{j}\Bigl[\frac{\partial z_{j}}{\partial y_{k}}dz_{j}\Bigr]$

Both Tensorflow and PyTorch perform the chain rule computation by *backpropagation* of the gradients through the neural network.
<img src="fig/example_graph_gradients.png" alt="drawing" width="900"/>

# Backpropagation in Tensorflow
- Tensorflow automatically calculates gradients of a graph
- "Automatically", meaning gradients are propagated to any point in the graph *given that the gradients of all operations in the graph have been clearly defined*
- For each operation, one must define how to send gradients back through the inputs, given the output gradients.  How is this done?

To calculate the derivative of the loss: 
- with respect to trainable variable $W_i$ (or input $x_i$)
- given output gradients $dy_{k}$

Compute the sum:

$dW_{i} = \frac{\partial L}{\partial W_{i}} = \sum_{k}\frac{\partial y_{k}}{\partial W_{i}}dy_{k}$

Or for input variable $x_i$, the gradient to send backward one step is:

$dx_{i} = \frac{\partial L}{\partial x_{i}} = \sum_{k}\frac{\partial y_{k}}{\partial x_{i}}dy_{k}$

## Exercises:
- Here we compute gradients for several operations used frequently in neural networks. 
- We'll start with one in which there is exactly one output for each input, and each output depends only on its corresponding input.

In [ ]:
import tensorflow as tf

### 1. The sigmoid function: $\sigma(x_i) = 1/(1+e^{-x_i})$

![example_sigmoid.png](fig/example_sigmoid.png)

Compute $dx$.

In [ ]:
# Define inputs and output gradients.
x = tf.constant([3.0, 4.0, 5.0])
dz = tf.constant([1.0, 2.0, 3.0])

In [ ]:
# Define the gradient.
def grad_sigmoid(x, dz):
    # (Add implementation here)
dx = grad_sigmoid(x,dz)
print(dx)

In [ ]:
# Compute the gradient with Tensorflow.
with tf.GradientTape() as g:
    g.watch(x)
    z = tf.sigmoid(x)
dx_tf = g.gradient(z, x, output_gradients=dz)
print(dx_tf)
print(dx == dx_tf)

$z_i = \sigma(x_i) = 1/(1+e^{-x_i})$

$\frac{\partial[\vec{\sigma}(\vec{x})]_i}{\partial x_j} = \frac{-1}{(1 + e^{-x_i})^2}\cdot e^{-x_i}\cdot(-1)\cdot\frac{\partial x_i}{\partial x_j}$

$= \frac{1}{(1 + e^{-z})}\cdot\frac{e^{-x_i}}{(1+e^{-x_i})}\cdot\delta_{ij}$

$= \sigma(x_{i})\cdot\frac{1 + e^{-x_i} - 1}{(1+e^{-x_i})}\cdot\delta_{ij}$

$= \sigma(x_{i})(1-\sigma(x_{i}))\delta_{ij}$

$dx_j = \sum_i dz_i \frac{\partial\sigma(x)_i}{\partial x_j}$

$ = \sum_i dz_i \sigma(x_{i})(1-\sigma(x_{i}))\delta_{ij}$

$ = dz_j \sigma(x_{j})(1-\sigma(x_{j}))$

or using the vector dot product,

$d\vec{x} = d\vec{z} \cdot d\vec{\sigma}(x)$

where 

$[d\vec{\sigma}(x)]_i = \sigma(x_{i})(1-\sigma(x_{i}))$

### Possible implementation of sigmoid gradient

In [ ]:
def grad_sigmoid(x, dz):
    return dz*tf.sigmoid(x)*(1-tf.sigmoid(x))

- What if each output now depends on more than one input?

### 2. The softmax function $\sigma_s(x_i) = e^{x_i}/\sum_{j}{e^{x_j}}$
![example_softmax.png](fig/example_softmax.png)

Compute $dx$.

In [ ]:
# Define inputs
x = tf.constant([3.0, 4.0, 5.0])
dz = tf.constant([1.0, 2.0, 3.0])

In [ ]:
# Define the gradient.
def grad_softmax(x, dz):
    # (Add implementation here)
dx = grad_softmax(x, dz)
print(dx)

In [ ]:
# Compute the gradient with Tensorflow.
with tf.GradientTape() as g:
    g.watch(x)
    z = tf.nn.softmax(x)
dx_tf = g.gradient(z, x, output_gradients=dz)
print(dx_tf)
print(dx == dx_tf)

$z_i = [\sigma_s(x)]_i = e^{x_i}/\sum_{j}{e^{x_j}}$

$\frac{\partial[\vec{\sigma_s}(\vec{x})]_i}{\partial x_j} = \frac{e^{x_i}\delta_{ij}}{\sum_k e^{x_k}} - \frac{e^{x_i}\cdot\sum_k e^{x_k}\delta_{jk}}{\bigl(\sum_{k}{e^{x_k}}\bigr)^2}$

$= \frac{e^{x_i}\delta_{ij}}{\sum_k e^{x_k}} - \frac{e^{x_i}e^{x_j}}{\bigl(\sum_{k}{e^{x_k}\bigr)^2}}$

$= [\sigma_s(x)]_i\delta_{ij} - [\sigma_s(x)]_i[\sigma_s(x)]_j$

$= [\sigma_s(x)]_i(\delta_{ij} - [\sigma_s(x)]_j)$

$dx_j = \sum_i dz_i \frac{\partial\sigma(x)_i}{\partial x_j}$

$ = \sum_i dz_i [\sigma_s(x)]_i(\delta_{ij} - [\sigma_s(x)]_j)$

$ = dz_j [\sigma_s(x)]_j - dz\cdot\sigma_s[\sigma_s(x)]_j$

$ = [\sigma_s(x)]_j(dz_j - d\vec{z}\cdot\vec{\sigma_s})$

### Possible implementation of the softmax gradient

In [ ]:
def grad_softmax(x,dz):
    return tf.nn.softmax(x)*(dz - tf.tensordot(tf.nn.softmax(x), dz, 1))

- So far, we have considered vector-valued inputs.
![tensor_inputs_outputs.png](fig/tensor_inputs_outputs.png)

- In general, we have tensor-valued inputs with multiple indices.
![vector_and_tensor.png](fig/vector_and_tensor.png)

### 3. Matrix multiplication: $z_{ij} = \sum_k{x_{ik}y_{kj}}$
![example_matrix_mult.png](fig/example_matrix_mult.png)

Compute $dx$.

In [ ]:
# Define inputs
x = tf.constant([[3.0, 4.0], [5.0, 6.0]])
y = tf.constant([[4.0, 5.0], [6.0, 7.0]])
dz = tf.constant([[1.0, 2.0], [3.0, 4.0]])

In [ ]:
# Define the gradient.
def grad_matmul(x, y, dz):
    # (Add implementation here)
dx = grad_matmul(x, y, dz)
print(dx)

In [ ]:
# Compute the gradient with Tensorflow.
with tf.GradientTape() as g:
    g.watch(x)
    z = tf.matmul(x, y)
dx_tf = g.gradient(z, x, output_gradients=dz)
print(dx_tf)
print(dx == dx_tf)

$z_{ij} = \sum_k{x_{ik}y_{kj}}$

$\frac{\partial z_{ij}}{\partial x_{lm}} = \sum_k{\delta_{il}\delta_{km}}y_{kj}$

$= \delta_{il}y_{mj}$

$dx_{lm} = \sum_{ij} dz_{ij} \frac{\partial z_{ij}}{\partial x_{lm}}$

$ = \sum_{ij} dz_{ij} \delta_{il}y_{mj}$

$ = \sum_{j}dz_{lj}y_{mj}$

$ = \sum_{j}dz_{lj}y^{T}_{jm}$

Note that this is the matrix multiplication:

$dx = dz \times y^{T}$

### Possible implementation of the matrix multiplication gradient

In [ ]:
def grad_matmul(x, y, dz):
    return tf.matmul(dz,tf.transpose(y))

- An operation such as convolution may require special care because it depends on several factors (strides, padding, filter sizes, etc.)
- We'll try a specific example, but note that the general case is more complex.

### 4. 2D Convolution $x * w$ ($k\times l$ filter): $z_{ij} = \sum_{kl}{w_{kl}x_{i+k,j+l}}$
![convolution.png](fig/example_convolution.png)
Compute $dw$. Assume for this exercise a stride of 1 in each dimension, and ignore padding.

In [ ]:
# 1. Use this method to perform the convolution.
def conv2d(x, w):
    return tf.nn.conv2d(tf.reshape(x,[1,x.shape[0],x.shape[1],1]),
                        tf.reshape(w,[w.shape[0],w.shape[1],1,1]), 
                        strides=[1,1],
                        padding="VALID")

In [ ]:
# 2. Define inputs
x = tf.constant([[3.0, 4.0, 5.0, 6.0], 
                 [4.0, 5.0, 6.0, 7.0],
                 [5.0, 6.0, 7.0, 8.0],
                 [6.0, 7.0, 8.0, 9.0]])
w = tf.constant([[1.0, 2.0], 
                 [3.0, 4.0]])
dz = tf.constant([[1.0, 1.0, 1.0], 
                  [2.0, 2.0, 2.0],
                  [3.0, 3.0, 3.0]])

In [ ]:
# 3. Define the gradient.
def grad_conv2d(x, w, dz):
    # (Add implementation here)
dx = grad_conv2d(x, w, dz)
dx = tf.reshape(dx, [w.shape[0],w.shape[1]]) # Reshape to remove channel and batch number dimensions.
print(dx)

In [ ]:
# 4. Compute the gradient with Tensorflow.
with tf.GradientTape() as g:
    g.watch(w)
    z = conv2d(x,w)
dx_tf = g.gradient(z, w, 
         output_gradients=tf.reshape(dz,[1,dz.shape[0],dz.shape[1],1]))
print(dx_tf)
print(dx == dx_tf)

$z_{ij} = \sum_{kl}{w_{kl}x_{i+k,j+l}}$

$\frac{\partial z_{ij}}{\partial w_{mn}} = \sum_{kl}{\delta_{km}\delta_{ln}}x_{i+k,j+l}$

$= x_{i+m,j+n}$

$dw_{mn} = \sum_{ij} dz_{ij} \frac{\partial z_{ij}}{\partial w_{mn}}$

$ = \sum_{ij} dz_{ij}x_{i+m,j+n}$

$ = \sum_{ij} dz_{ij}x_{m+i,n+j}$

Note that this is the convolution:

$dw = x * dz$

### Possible implementation of the 2D convolution gradient:

In [ ]:
def grad_conv2d(x, w, dz):
    return conv2d(x, dz)